# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df = pd.read_csv('Final_Project_datasets/nhl.csv')
cities = pd.read_html('Final_Project_datasets/wikipedia_data.html')[1]
cities = cities.iloc[:-1,[0,3,5,6,7,8]]

#Function below will find all words in the column Team Name which start with a capital letter from the city df.
#If the length of the result is zero, it means no teams for that city were present in the format described above. We will
#place a zero for now for these entries to make dropping these values later easy. 
def team_separator(row):
    pattern = '([A-Z][a-z]*)'
    row['Team Name'] = re.findall(pattern, row['Team Name'])
    if len(row['Team Name']) == 0:
        row['Team Name'] = 0
    return row

    
def nhl_correlation(): 

    #To carry out the necessary data cleaning, restrict the df to only the relevant columns, then remove all the bits inbetween
    #the square brackets using regex. After, rename the team column to something more relevant, and apply the team_separator
    #function to the cleaned df. Drop any unnecessary values (cities with no teams), and make sure each row has a unique team 
    #name. Finally, we will set the team name as the index.
    
    cities_nhl_clean_df = cities.iloc[:,[0, 1, -1]]
    cities_nhl_clean_df = (cities_nhl_clean_df
                            .replace(to_replace = '\[.*\]', value = '', regex = True)
                            .rename(columns = {'NHL':'Team Name'})
                            .apply(team_separator, axis = 1)
                          )
    cities_nhl_clean_df = cities_nhl_clean_df[cities_nhl_clean_df['Team Name'] != 0]
    cities_nhl_clean_df = cities_nhl_clean_df.explode('Team Name')
    cities_nhl_clean_df.set_index('Team Name', inplace = True)
    
    
    #Next, we will import the relevant columns from the nhl csv file for the year 2018, and rename the column of teams. After,
    #we will use regex to simplify the team names into one item, as done in the cities df. Then only carry the last item
    #since this is the only identifier which is necessary
    sports_nhl_clean_df = nhl_df[nhl_df['year'] == 2018]
    sports_nhl_clean_df = sports_nhl_clean_df.iloc[:, :4]
    sports_nhl_clean_df.rename(columns = {'team':'Team Name'}, inplace = True)

    sports_nhl_clean_df.apply(team_separator, axis = 1)
    sports_nhl_clean_df['Team Name'] = sports_nhl_clean_df.apply(lambda x: x['Team Name'][-1], axis = 1)
    sports_nhl_clean_df.set_index('Team Name', inplace = True)
    

    print('Teams in cities df which are not in sports df:', \
          cities_nhl_clean_df[~cities_nhl_clean_df.index.isin(sports_nhl_clean_df.index)])
    
    #From the above we can say that 4 teams haven't matched from cities df to sports df - this was due to a simplification made
    #in the cleaning process in assuming that the unique identifier for teams was only the last word. The next step will be to 
    #manually fix these errors.
    cities_nhl_clean_df.rename(index={
        'Wings':'Red Wings',
        'Leafs':'Maple Leafs',
        'Jackets':'Blue Jackets',
        'Knights':'Golden Knights'
    }, 
    inplace = True)
    
    sports_nhl_clean_df.rename(index={
        'Wings':'Red Wings',
        'Leafs':'Maple Leafs',
        'Jackets':'Blue Jackets',
        'Knights':'Golden Knights'
    }, 
    inplace = True)
    
    cities_nhl_clean_df.drop(['Red', 'Maple', 'Blue', 'Golden'], inplace = True)
    
    #Now the next step will be to merge both of the dataframes according to the team names. 
    merged_df = pd.merge(cities_nhl_clean_df, sports_nhl_clean_df, how = 'inner', left_index = True, right_index = True)
    
    #To this merged df, lets add another column which will represent the win/loss ratio for every NHL team.
    merged_df['L'] = pd.to_numeric(merged_df['L'])
    merged_df['W'] = pd.to_numeric(merged_df['W'])
    merged_df['Win/Loss Ratio'] = merged_df['W']/(merged_df['L']+merged_df['W'])
    merged_df['Win/Loss Ratio'] = pd.to_numeric(merged_df['Win/Loss Ratio'])
    merged_df['Population (2016 est.)[8]'] = pd.to_numeric(merged_df['Population (2016 est.)[8]'])

    
    #Now lets group teams by Metropolitan Areas, and find the mean win/loss ratio for teams within the same area.
    final_df = merged_df.groupby(['Metropolitan area'], as_index = True)[['Win/Loss Ratio', 'Population (2016 est.)[8]']].mean()
    
    print(final_df)
    
    population_by_region = pd.to_numeric(final_df['Population (2016 est.)[8]']) # pass in metropolitan area population from cities
    win_loss_by_region = pd.to_numeric(final_df['Win/Loss Ratio']) # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nhl_correlation()

Teams in cities df which are not in sports df:           Metropolitan area Population (2016 est.)[8]
Team Name                                            
Red                 Detroit                   4297617
Maple               Toronto                   5928040
Blue               Columbus                   2041520
Golden            Las Vegas                   2155664
                        Win/Loss Ratio  Population (2016 est.)[8]
Metropolitan area                                                
Boston                        0.714286                  4794447.0
Buffalo                       0.357143                  1132804.0
Calgary                       0.513889                  1392609.0
Chicago                       0.458333                  9512999.0
Columbus                      0.600000                  2041520.0
Dallas–Fort Worth             0.567568                  7233323.0
Denver                        0.589041                  2853077.0
Detroit                       0.434

0.012486162921209909

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df = pd.read_csv('Final_Project_datasets/nba.csv')
cities = pd.read_html('Final_Project_datasets/wikipedia_data.html')[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]


#Added an if branch to deal with the edge case of Philadelphia 76ers wherein second part starts with digit not capital letter.
def team_separator(row):
    pattern = '([A-Z][a-z]*)'
    row['Team Name'] = re.findall(pattern, row['Team Name'])
    if len(row['Team Name']) == 0:
        row['Team Name'] = 0
    if 'Metropolitan area' in row: 
        if row['Metropolitan area'] == 'Philadelphia':
            row['Team Name'] = '76ers'
    return row

def nba_correlation():
    #To carry out the necessary data cleaning, restrict the df to only the relevant columns, then remove all the bits inbetween
    #the square brackets using regex. After, rename the team column to something more relevant, and apply the team_separator
    #function to the cleaned df. Drop any unnecessary values (cities with no teams), and make sure each row has a unique team 
    #name. Finally, we will set the team name as the index.
    
    cities_nba_clean_df = cities.iloc[:,[0, 1, -2]]
    print(cities)
    cities_nba_clean_df = (cities_nba_clean_df
                            .replace(to_replace = '\[.*\]', value = '', regex = True)
                            .rename(columns = {'NBA':'Team Name'})
                            .apply(team_separator, axis = 1)
                          )
    cities_nba_clean_df = cities_nba_clean_df[cities_nba_clean_df['Team Name'] != 0]
    cities_nba_clean_df = cities_nba_clean_df.explode('Team Name')
    cities_nba_clean_df.set_index('Team Name', inplace = True)
    cities_nba_clean_df.rename(index={
        'Trail':'Blazers',
    }, 
    inplace = True)
    print(cities_nba_clean_df)
    
    #Next, we will import the relevant columns from the nba csv file for the year 2018, and rename the column of teams. After,
    #we will use regex to simplify the team names into one item, as done in the cities df. Then only carry the last item
    #since this is the only identifier which is necessary
    sports_nba_clean_df = nba_df[nba_df['year'] == 2018]
    sports_nba_clean_df = sports_nba_clean_df.iloc[:, :3]
    sports_nba_clean_df.rename(columns = {'team':'Team Name'}, inplace = True)

    sports_nba_clean_df.apply(team_separator, axis = 1)
    sports_nba_clean_df['Team Name'] = sports_nba_clean_df.apply(lambda x: x['Team Name'][-1], axis = 1)
    sports_nba_clean_df.set_index('Team Name', inplace = True)
    sports_nba_clean_df.rename(index={
        'Philadelphia':'76ers',
    }, 
    inplace = True)
    print(sports_nba_clean_df)
    
    print('Teams in cities df which are not in sports df:', \
          cities_nba_clean_df[~cities_nba_clean_df.index.isin(sports_nba_clean_df.index)])
    
    
    #Now the next step will be to merge both of the dataframes according to the team names. 
    merged_df = pd.merge(cities_nba_clean_df, sports_nba_clean_df, how = 'inner', left_index = True, right_index = True)
    
    #To this merged df, lets add another column which will represent the win/loss ratio for every NHL team.
    merged_df['L'] = pd.to_numeric(merged_df['L'])
    merged_df['W'] = pd.to_numeric(merged_df['W'])
    merged_df['Win/Loss Ratio'] = merged_df['W']/(merged_df['L']+merged_df['W'])
    merged_df['Win/Loss Ratio'] = pd.to_numeric(merged_df['Win/Loss Ratio'])
    merged_df['Population (2016 est.)[8]'] = pd.to_numeric(merged_df['Population (2016 est.)[8]'])
    
    #Now lets group teams by Metropolitan Areas, and find the mean win/loss ratio for teams within the same area.
    final_df = merged_df.groupby(['Metropolitan area'], as_index = True)[['Win/Loss Ratio', 'Population (2016 est.)[8]']].mean()
    
    population_by_region = final_df['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = final_df['Win/Loss Ratio'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nba_correlation()

         Metropolitan area Population (2016 est.)[8]                   NFL  \
0            New York City                  20153634    GiantsJets[note 1]   
1              Los Angeles                  13310447  RamsChargers[note 4]   
2   San Francisco Bay Area                   6657982  49ersRaiders[note 6]   
3                  Chicago                   9512999         Bears[note 8]   
4        Dallas–Fort Worth                   7233323               Cowboys   
5         Washington, D.C.                   6131977              Redskins   
6             Philadelphia                   6070500                Eagles   
7                   Boston                   4794447     Patriots[note 14]   
8   Minneapolis–Saint Paul                   3551036               Vikings   
9                   Denver                   2853077               Broncos   
10   Miami–Fort Lauderdale                   6066387              Dolphins   
11                 Phoenix                   4661537            

-0.17657160252844611

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [6]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df = pd.read_csv('Final_Project_datasets/mlb.csv')
cities = pd.read_html('Final_Project_datasets/wikipedia_data.html')[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

#Added an if branch to deal with the edge case of Philadelphia 76ers wherein second part starts with digit not capital letter.
def team_separator(row):
    pattern = '([A-Z][a-z]*)'
    row['Team Name'] = re.findall(pattern, row['Team Name'])
    if len(row['Team Name']) == 0:
        row['Team Name'] = 0
    return row

def sox_organiser(row):
    if row['Metropolitan area'] == 'Chicago':
        if row['Team Name'] == 'Sox':
            row['Team Name'] = 'White Sox'
    elif row['Metropolitan area'] == 'Boston':
        row['Team Name'] = 'Red Sox'
    return row

def sports_organiser(row):
    if row[-1] == 'Sox':
        if row[-2] == 'White':
            return 'White Sox'
        else:
            return 'Red Sox'
    else:
        return row[-1]

def mlb_correlation(): 
    #To carry out the necessary data cleaning, restrict the df to only the relevant columns, then remove all the bits inbetween
    #the square brackets using regex. After, rename the team column to something more relevant, and apply the team_separator
    #function to the cleaned df. Drop any unnecessary values (cities with no teams), and make sure each row has a unique team 
    #name. Finally, we will set the team name as the index.
    
    cities_mlb_clean_df = cities.iloc[:,[0, 1, -3]]
    cities_mlb_clean_df = (cities_mlb_clean_df
                            .replace(to_replace = '\[.*\]', value = '', regex = True)
                            .rename(columns = {'MLB':'Team Name'})
                            .apply(team_separator, axis = 1)
                          )
    cities_mlb_clean_df = cities_mlb_clean_df[cities_mlb_clean_df['Team Name'] != 0]
    cities_mlb_clean_df = (cities_mlb_clean_df
                           .explode('Team Name')
                           .reset_index(drop = True)
                          )
    
    cities_mlb_clean_df.apply(sox_organiser, axis = 1)
    cities_mlb_clean_df = cities_mlb_clean_df.drop([cities_mlb_clean_df.iloc[7].name,
                                                   cities_mlb_clean_df.iloc[12].name,
                                                   cities_mlb_clean_df.iloc[19].name]
                                                  )
    cities_mlb_clean_df.set_index('Team Name', inplace = True)
    
    #Next, we will import the relevant columns from the nba csv file for the year 2018, and rename the column of teams. After,
    #we will use regex to simplify the team names into one item, as done in the cities df. Then only carry the last item
    #since this is the only identifier which is necessary
    sports_mlb_clean_df = mlb_df[mlb_df['year'] == 2018]
    sports_mlb_clean_df = sports_mlb_clean_df.iloc[:, :3]
    sports_mlb_clean_df = (sports_mlb_clean_df
                           .rename(columns = {'team':'Team Name'})
                           .apply(team_separator, axis = 1)
                          )

    sports_mlb_clean_df['Team Name'] = sports_mlb_clean_df['Team Name'].apply(sports_organiser)
    sports_mlb_clean_df.set_index('Team Name', inplace = True)

    #Now the next step will be to merge both of the dataframes according to the team names. 
    merged_df = pd.merge(cities_mlb_clean_df, sports_mlb_clean_df, how = 'inner', left_index = True, right_index = True)
    
    #To this merged df, lets add another column which will represent the win/loss ratio for every NHL team.
    merged_df['L'] = pd.to_numeric(merged_df['L'])
    merged_df['W'] = pd.to_numeric(merged_df['W'])
    merged_df['Win/Loss Ratio'] = merged_df['W']/(merged_df['L']+merged_df['W'])
    merged_df['Win/Loss Ratio'] = pd.to_numeric(merged_df['Win/Loss Ratio'])
    merged_df['Population (2016 est.)[8]'] = pd.to_numeric(merged_df['Population (2016 est.)[8]'])
    
    #Now lets group teams by Metropolitan Areas, and find the mean win/loss ratio for teams within the same area.
    final_df = merged_df.groupby(['Metropolitan area'], as_index = True)[['Win/Loss Ratio', 'Population (2016 est.)[8]']].mean()
    
    print(final_df)
    population_by_region = final_df['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = final_df['Win/Loss Ratio'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

mlb_correlation()

                        Win/Loss Ratio  Population (2016 est.)[8]
Metropolitan area                                                
Atlanta                       0.555556                  5789700.0
Baltimore                     0.290123                  2798886.0
Boston                        0.666667                  4794447.0
Chicago                       0.482769                  9512999.0
Cincinnati                    0.413580                  2165139.0
Cleveland                     0.561728                  2055612.0
Dallas–Fort Worth             0.413580                  7233323.0
Denver                        0.558282                  2853077.0
Detroit                       0.395062                  4297617.0
Houston                       0.635802                  6772470.0
Kansas City                   0.358025                  2104509.0
Los Angeles                   0.529122                 13310447.0
Miami–Fort Lauderdale         0.391304                  6066387.0
Milwaukee 

0.15027698302669307

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [7]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv('Final_Project_datasets/nfl.csv')
cities=pd.read_html('Final_Project_datasets/wikipedia_data.html')[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_organiser(row):
    if row[-1] == 'Sox':
        if row[-2] == 'White':
            return 'White Sox'
        else:
            return 'Red Sox'
    else:
        return row[-1]

#Added an if branch to deal with the edge case of Philadelphia 76ers wherein second part starts with digit not capital letter.
def team_separator(row):
    pattern = '([A-Z][a-z]*)|([\d][\d][a-z]*)'
    row['Team Name'] = re.findall(pattern, row['Team Name'])
    if len(row['Team Name']) == 0:
        row['Team Name'] = 0
    return row

#Added this function to fix the exception to regex function posed by the team San Francisco 49ers
def exception_handler(row):
    if row[1] == '49ers':
        return '49ers'
    else:
        return row[0]

def nfl_correlation(): 
    #To carry out the necessary data cleaning, restrict the df to only the relevant columns, then remove all the bits inbetween
    #the square brackets using regex. After, rename the team column to something more relevant, and apply the team_separator
    #function to the cleaned df. Drop any unnecessary values (cities with no teams), and make sure each row has a unique team 
    #name. Finally, we will set the team name as the index.
    cities_nfl_clean_df = cities.iloc[:,[0, 1, -4]]
    cities_nfl_clean_df = (cities_nfl_clean_df
                            .replace(to_replace = '\[.*\]', value = '', regex = True)
                            .rename(columns = {'NFL':'Team Name'})
                            .apply(team_separator, axis = 1)
                          )
    cities_nfl_clean_df = cities_nfl_clean_df[cities_nfl_clean_df['Team Name'] != 0]
    cities_nfl_clean_df = (cities_nfl_clean_df
                           .explode('Team Name')
                           .reset_index(drop = True)
                          )
    cities_nfl_clean_df['Team Name'] = cities_nfl_clean_df['Team Name'].apply(exception_handler)
    cities_nfl_clean_df.set_index('Team Name', inplace = True)
    
    #Next, we will import the relevant columns from the nba csv file for the year 2018, and rename the column of teams. After,
    #we will use regex to simplify the team names into one item, as done in the cities df. Then only carry the last item
    #since this is the only identifier which is necessary
    sports_nfl_clean_df = nfl_df[nfl_df['year'] == 2018]
    sports_nfl_clean_df = sports_nfl_clean_df.iloc[:, [1,-4,-2]]
    sports_nfl_clean_df = (sports_nfl_clean_df
                           .rename(columns = {'team':'Team Name'})
                           .apply(team_separator, axis = 1)
                          )

    sports_nfl_clean_df['Team Name'] = sports_nfl_clean_df['Team Name'].apply(sports_organiser)
    sports_nfl_clean_df['Team Name'] = sports_nfl_clean_df['Team Name'].apply(exception_handler)
    sports_nfl_clean_df.set_index('Team Name', inplace = True)
    
    #Now the next step will be to merge both of the dataframes according to the team names. 
    merged_df = pd.merge(cities_nfl_clean_df, sports_nfl_clean_df, how = 'inner', left_index = True, right_index = True)
    
    #To this merged df, lets add another column which will represent the win/loss ratio for every NHL team.
    merged_df['L'] = pd.to_numeric(merged_df['L'])
    merged_df['W'] = pd.to_numeric(merged_df['W'])
    merged_df['Win/Loss Ratio'] = merged_df['W']/(merged_df['L']+merged_df['W'])
    merged_df['Win/Loss Ratio'] = pd.to_numeric(merged_df['Win/Loss Ratio'])
    merged_df['Population (2016 est.)[8]'] = pd.to_numeric(merged_df['Population (2016 est.)[8]'])
    
    #Now lets group teams by Metropolitan Areas, and find the mean win/loss ratio for teams within the same area.
    final_df = merged_df.groupby(['Metropolitan area'], as_index = True)[['Win/Loss Ratio', 'Population (2016 est.)[8]']].mean()
    
    population_by_region = final_df['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = final_df['Win/Loss Ratio'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nfl_correlation()

0.004922112149349456

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NFL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values